Import

In [6]:
from ..dataset import PerturbationDataset
from ..models.scgen import scGenModel

hello world


Data processing

Modelling

In [5]:
# standardize train and test data to the method specific format
train_data = PerturbationDataset(train_adata)
test_data = PerturbationDataset(test_adata)

# define the models for benchmarking
scgen_model = scGenModel()
scgen_model.train(train_data) # default hyper-parameters
scgen_model.save("path/to/save/model")


hello world


Prediction

# load and predict the test data
scgen_model.load("path/to/save/model")
predicted_data = scgen_model.predict(test_data)

Evaluation